In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib as mpl

from scipy.optimize import curve_fit

In [ ]:
mpl.rc("font", size=14)

In [ ]:
import os
from DataHandling import ReadBinary, GetSBCEvent
from LAr10Ana.SiPMPulses import SiPMPulses
from LAr10Ana.PhotonT0 import PhotonT0

path = "/bluearc/storage/SBC-22-caendata"
file = "202206201045.bin"

filename = os.path.join(path, file)

data = ReadBinary.ReadBlock(filename)
sipm_data = SiPMPulses(data)
comb_data = PhotonT0(sipm_data)

In [ ]:
bins = np.linspace(0, 0.05, 101)
centers = (bins[:-1] + bins[1:]) / 2

N,_,_ = plt.hist(sipm_data["hit_amp"].T, histtype="step", bins=bins, label=["Channel %i" % i for i in range(sipm_data["hit_amp"].shape[0])])

plt.xlabel("Hit Amplitude [V]")
plt.ylabel("# Hits")
plt.legend()
plt.tight_layout()
plt.savefig("figures/sipm_amplitudes.png")

In [ ]:
def gauss(V, A, mu, sigma):
    return A * np.exp(-(V-mu)**2/(2*sigma**2))

def ngauss(V, *p):
    assert(len(p) % 3 == 0)
    ngaus = len(p) // 3
    return np.sum([gauss(V, *p[i*3:(i+1)*3]) for i in range(ngaus)], axis=0)

In [ ]:
NFIT = 5

mu_p0 = np.array([0.005*(i+1) for i in range(NFIT)])
sig_p0 = np.array([0.0005]*NFIT)
A_p0 = np.array([1.2e4/2**i for i in range(NFIT)])

p0 = sum([[A_p0[i], mu_p0[i], sig_p0[i]] for i in range(NFIT)], [])

mu_lo = mu_p0 - 0.0025
mu_hi = mu_p0 + 0.0025

sig_lo = np.array([0.00025]*NFIT)
sig_hi = np.array([0.00100]*NFIT)

A_lo = A_p0/2
A_hi = A_p0*2

lo = sum([[A_lo[i], mu_lo[i], sig_lo[i]] for i in range(NFIT)], [])
hi = sum([[A_hi[i], mu_hi[i], sig_hi[i]] for i in range(NFIT)], [])

In [ ]:
N, _ = np.histogram(sipm_data["hit_amp"].T, bins=bins)

In [ ]:
sipm_data["hit_amp"].shape

In [ ]:
popts = []

for i_sipm in range(3):
    where_fit = (centers > 0.003) & (centers < 0.027)
    popt, perr = curve_fit(ngauss, centers[where_fit], N[i_sipm][where_fit], p0=p0, bounds=(lo, hi))
    
    popts.append(popt)

#     amps = popt[::3]
#     peaks = popt[1::3]
#     widths = popt[2::3]

In [ ]:
for i_sipm in range(3):
    plt.figure(i_sipm)
    
    plt.plot(centers[where_fit], N[i_sipm][where_fit])

    plt.plot(centers[where_fit], ngauss(centers[where_fit], *p0))
    plt.plot(centers[where_fit], ngauss(centers[where_fit], *popts[i_sipm]))

In [ ]:
for i_sipm in range(3):
    plt.figure(i_sipm)
    popt = popts[i_sipm]
    peaks = popt[1::3]
    
    npe = np.arange(1, len(peaks)+1)
    m, b = np.polyfit(npe, peaks, 1)

    plt.plot(npe, peaks)
    plt.plot(npe, npe*m+b)

    print(i_sipm, m, b)